Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. 

Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. 

Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [120]:
import pandas as pd

# знакомимся с датафреймом
films_rate = pd.read_csv('ratings.csv', sep=',')
films_rate.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [131]:
# Прооверим, сколько у нас уникальных пользователей 
print(len(films_rate['userId'].unique()))

671


In [122]:
# Подготовим столбец с помощью которого будем подсчитывать количество оценок каждого из пользоватлей
films_rate['counter']= 1

In [123]:
films = films_rate.groupby(['userId','movieId']).sum()
films.head(5)

rating   timestamp  counter
userId movieId                             
1      31          2.5  1260759144        1
       1029        3.0  1260759179        1
       1061        3.0  1260759182        1
       1129        2.0  1260759185        1
       1172        4.0  1260759205        1

In [124]:
films_pivot = films.pivot_table(index='userId', columns='rating', values='counter', aggfunc='sum').fillna(0)
films_pivot.head(5)

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
1,0.0,2.0,0.0,7.0,3.0,4.0,1.0,3.0,0.0,0.0
2,0.0,2.0,0.0,4.0,0.0,36.0,0.0,23.0,0.0,11.0
3,0.0,0.0,0.0,1.0,3.0,18.0,9.0,11.0,4.0,5.0
4,0.0,5.0,0.0,5.0,0.0,23.0,0.0,52.0,0.0,119.0
5,0.0,0.0,1.0,0.0,3.0,3.0,27.0,42.0,19.0,5.0


In [125]:
# Рассчитаем,сколько оценок дал каждый (уникальный) пользователь
films_pivot['counter'] = films_pivot.sum(axis=1)
films_pivot.head(5)

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,counter
userId,,,,,,,,,,,
1,0.0,2.0,0.0,7.0,3.0,4.0,1.0,3.0,0.0,0.0,20.0
2,0.0,2.0,0.0,4.0,0.0,36.0,0.0,23.0,0.0,11.0,76.0
3,0.0,0.0,0.0,1.0,3.0,18.0,9.0,11.0,4.0,5.0,51.0
4,0.0,5.0,0.0,5.0,0.0,23.0,0.0,52.0,0.0,119.0,204.0
5,0.0,0.0,1.0,0.0,3.0,3.0,27.0,42.0,19.0,5.0,100.0


In [126]:
# Сделаем фильтр на строки со значением количества оценок больше 100
rating_counter = float(100)
films_rating_counter=films_pivot[films_pivot.counter > rating_counter]
films_rating_counter.head(5)


rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,counter
userId,,,,,,,,,,,
4,0.0,5.0,0.0,5.0,0.0,23.0,0.0,52.0,0.0,119.0,204.0
8,1.0,0.0,0.0,3.0,4.0,13.0,26.0,35.0,16.0,18.0,116.0
15,83.0,274.0,128.0,217.0,141.0,312.0,146.0,251.0,56.0,92.0,1700.0
17,16.0,6.0,4.0,20.0,22.0,30.0,45.0,64.0,79.0,77.0,363.0
19,0.0,15.0,0.0,14.0,0.0,195.0,0.0,128.0,0.0,71.0,423.0


In [127]:
films_lots_ratings = films_rating_counter.filter(items=['userId','counter'])
films_lots_ratings.head(5)

rating,counter
userId,
4,204.0
8,116.0
15,1700.0
17,363.0
19,423.0


In [141]:
films_lots_ratings.reset_index().head(5)

rating,userId,counter
0,4,204.0
1,8,116.0
2,15,1700.0
3,17,363.0
4,19,423.0


In [142]:
#Склеим 2 таблицы между собой по методу left join (чтобы у нас остались только те значения userId, где оценок более 100)
films_all= films_lots_ratings.merge(films_rate, how ='left', on ='userId')
films_all

,userId,counter_x,movieId,rating,timestamp,counter_y
0,4,204.0,10,4.0,949810645,1
1,4,204.0,34,5.0,949919556,1
2,4,204.0,112,5.0,949810582,1
3,4,204.0,141,5.0,949919681,1
4,4,204.0,153,4.0,949811346,1
...,...,...,...,...,...,...
80482,671,115.0,6268,2.5,1065579370,1
80483,671,115.0,6269,4.0,1065149201,1
80484,671,115.0,6365,4.0,1070940363,1
80485,671,115.0,6385,2.5,1070979663,1


In [130]:
# Узнаем сколько уникальных пользователей поставили оценки более 100 раз
print(len(films_all['userId'].unique()))


258


In [133]:
# Можем сделать вывод, что менее половины пользователей проставляли более 100 оценок
# Отфильтруем необходимые нам столбцы для наглядности
films_all_filtered = films_all.filter(items=['userId','timestamp'])
films_all_filtered

,userId,timestamp
0,4,949810645
1,4,949919556
2,4,949810582
3,4,949919681
4,4,949811346
...,...,...
80482,671,1065579370
80483,671,1065149201
80484,671,1070940363
80485,671,1070979663


In [143]:
# Саггрегируем min и max, чтобы, используя разницу минимального и максимального занчения timestamp, вычислить время жизни пользователей для каждой оценки
films_ratings_grouped = films_all_filtered.groupby('userId').agg([min, max])
films_ratings_grouped.head()

timestamp            
               min         max
userId                        
4        949778714   949982274
8       1154389340  1154474527
15       997937239  1469330735
17      1127468587  1127476640
19       855190091   855195373

In [140]:
# Находим lifetime
films_ratings_grouped['lifetime'] = films_ratings_grouped['timestamp']['max'] - films_ratings_grouped['timestamp']['min']
films_ratings_grouped.head()
# Вывожу значение userId из индекса в столбец
films_ratings_grouped.reset_index().head()

userId   timestamp               lifetime
                 min         max           
0      4   949778714   949982274     203560
1      8  1154389340  1154474527      85187
2     15   997937239  1469330735  471393496
3     17  1127468587  1127476640       8053
4     19   855190091   855195373       5282